In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/news-headlines-dataset-for-sarcasm-detection/Sarcasm_Headlines_Dataset_v2.json
/kaggle/input/news-headlines-dataset-for-sarcasm-detection/Sarcasm_Headlines_Dataset.json


In [2]:
#Necessary packages
import pandas as pd
import numpy as np
import os
import json
from sklearn.model_selection import train_test_split as tts
from sklearn.model_selection import cross_val_score as cvs
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
import keras
import tensorflow as tf
from keras import models
from keras import layers
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.losses import MeanSquaredError as mse
import sklearn.metrics
from sklearn.metrics import confusion_matrix as cm
from sklearn.metrics import precision_score, recall_score

In [3]:
#Getting pathnames for each file in the input folder
for dirname, _, filenames in os.walk('/kaggle/input/news-headlines-dataset-for-sarcasm-detection'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

#Function to parse data
def parse_data(file):
    for l in open(file,'r'):
        yield json.loads(l)

#Taking in the data in one of the json files (I'm using the slightly larger one)
data = list(parse_data("/kaggle/input/news-headlines-dataset-for-sarcasm-detection/Sarcasm_Headlines_Dataset_v2.json"))
data[0]

/kaggle/input/news-headlines-dataset-for-sarcasm-detection/Sarcasm_Headlines_Dataset_v2.json
/kaggle/input/news-headlines-dataset-for-sarcasm-detection/Sarcasm_Headlines_Dataset.json


{'is_sarcastic': 1,
 'headline': 'thirtysomething scientists unveil doomsday clock of hair loss',
 'article_link': 'https://www.theonion.com/thirtysomething-scientists-unveil-doomsday-clock-of-hai-1819586205'}


We need to separate the data from this list into X and y
Our X is going to be a vectorized list of words from the headline.
Our y is going to be "is_sarcastic".
To do this, we're going to use some NLP packages

In [4]:
#Creating our X variable
vectorizer = TfidfVectorizer(max_features=50, use_idf=False)
headlines = [i['headline'] for i in data]
X = vectorizer.fit_transform(headlines).toarray()

#Creating our y variable
y = np.ravel([i['is_sarcastic'] for i in data])

#Creating a train and test split
X_train, X_test, y_train, y_test = tts(X, y, test_size = 0.2, random_state = 1693)

In [5]:
#Now we're going to build the model with its layers

#Initialize the model
model = Sequential([
    # first layer
    Dense(units=24, input_shape=(50,), activation='softmax'),
    # second layer
    Dense(12, activation = 'softmax'),
    # third layer
    Dense(8, activation = 'softmax'),
    # output layer
    Dense(1, activation = 'sigmoid')
])

In [6]:
#Now we're going to compile the model
#Our loss function is binary crossentropy
#Our optimizer is adam

model.compile(loss = 'binary_crossentropy', 
              optimizer = 'adam',
              metrics = ['accuracy', 'mse'])

#We're going to also fit the model
#We're going to do 20 epochs
#The batch size will be 224 to get ~100 iterations per epoch
model.fit(X_train, y_train, epochs = 20,
          batch_size = 224, verbose = 1)

Epoch 1/20
103/103 [==============================] - 1s 2ms/step - loss: 0.6930 - accuracy: 0.5240 - mse: 0.2499
Epoch 2/20
103/103 [==============================] - 0s 2ms/step - loss: 0.6926 - accuracy: 0.5171 - mse: 0.2497
Epoch 3/20
103/103 [==============================] - 0s 2ms/step - loss: 0.6919 - accuracy: 0.5225 - mse: 0.2494
Epoch 4/20
103/103 [==============================] - 0s 2ms/step - loss: 0.6907 - accuracy: 0.5261 - mse: 0.2488
Epoch 5/20
103/103 [==============================] - 0s 2ms/step - loss: 0.6876 - accuracy: 0.5248 - mse: 0.2472
Epoch 6/20
103/103 [==============================] - 0s 2ms/step - loss: 0.6781 - accuracy: 0.5918 - mse: 0.2425
Epoch 7/20
103/103 [==============================] - 0s 2ms/step - loss: 0.6558 - accuracy: 0.7048 - mse: 0.2314
Epoch 8/20
103/103 [==============================] - 0s 2ms/step - loss: 0.6254 - accuracy: 0.7118 - mse: 0.2166
Epoch 9/20
103/103 [==============================] - 0s 2ms/step - loss: 0.5936 - accur

In [7]:
#Next, we'll test the model on the test dataset we set aside

#Prediction on the X_test data, round each to an integer (either 0 or 1)
y_pred = np.around(model.predict(X_test))

#We're going to now look at the accuracy and loss
score = model.evaluate(X_test, y_test, verbose=1)
print(score)

#We'll print precision and recall too
print(f"Precision: {precision_score(y_test, y_pred)}")
print(f"Recall: {recall_score(y_test, y_pred)}")

179/179 [==============================] - 0s 957us/step - loss: 0.5532 - accuracy: 0.7098 - mse: 0.1869
[0.553242027759552, 0.7098183035850525, 0.18690022826194763]
Precision: 0.6560509554140127
Recall: 0.804089219330855


In [8]:
#Now we're going to make a confusion matri
#The rows are the known labels, the columns are the predicted labels
matrix = cm(y_test, y_pred)
df = pd.DataFrame(columns = ['', 'is_sarcastic', 'not_sarcastic'])
df.loc[len(df)] = ['is_sarcastic', matrix[0][0], matrix[0][1]]
df.loc[len(df)] = ['not_sarcastic', matrix[1][0], matrix[1][1]]
print(df)

                 is_sarcastic not_sarcastic
0   is_sarcastic         1900          1134
1  not_sarcastic          527          2163
